In [ ]:
import pandas as pd
import numpy as np

Load the dataset. The file `BL-Flickr-Images-Book.csv` is a CSV file containing information about books from the British Library.

In [ ]:
df = pd.read_csv('BL-Flickr-Images-Book.csv')
df.head()

Inspect the columns list.

In [ ]:
df.columns

Drop some columns that are not of interest.

In [ ]:
to_drop = ['Edition Statement',
           'Corporate Author', 
           'Corporate Contributors', 
           'Former owner', 
           'Engraver', 
           'Contributors', 
           'Issuance type', 
           'Shelfmarks']
df.drop(to_drop, inplace=True, axis=1)
df.head()

Check if the `Identifier` column contains entirely unique values.

In [ ]:
df['Identifier'].is_unique

Set the index to be the `Identifier` column.

In [ ]:
df = df.set_index('Identifier')
df.head()

We can retrieve columns by referencing the index using `.loc[ ... ]`.

In [ ]:
df.loc[206]

If we look at one particular colum, `Date of Publication`, we can see that there are a few different ways that dates are expressed in the dataset.

In [ ]:
df.loc[1905:, 'Date of Publication']  # this selects record ID 1905 to the end of the dataset, for the given column only

We can use the regular expression `^(\d{4})` to select only the publication year from the data. We can test it on a copy of just that column.

In [ ]:
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.loc[1905:]

Now we know that it works, we can reset the column contents using the cleaned version.

In [ ]:
df['Date of Publication'] = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
df.loc[1905:]

You probably noticed that there's a lot of null values (`NaN` - Not a Number) in the column. We can calcuate the percentage of null values by summing up the null values and dividing by the number of rows in the dataset and multiply by 100.

In [ ]:
df['Date of Publication'].isnull().sum() / len(df) * 100

Let's take a look at another colum now, `Place of Publication`.

In [ ]:
df['Place of Publication'].head(10)

We can see, there are also problem with this column! By inspecting these 10 rows, we can see that places that are supposed to be London and Oxford might be formatted differently. Let's also inspect two specific records.

In [ ]:
df.loc[4157862]

In [ ]:
df.loc[4159587]

We can see from these records that these two books were published in the same place, but one `Place of Publication` entry uses hyphens while the other does not.

One strategy to fix all of these is to look for all instances where `London` apppears inside the text and replace it specifically with the normalised text as only `London`, and the same for `Oxford`. We can also try and clean up the hyphens by replacing those with a single space.

In [ ]:
df['Place of Publication'] = np.where(df['Place of Publication'].str.contains('London'), 'London', 
                                 np.where(df['Place of Publication'].str.contains('Oxford'), 'Oxford', 
                                     df['Place of Publication'].str.replace('-', ' ')))

In [ ]:
df.head(10)

At this point, we have cleaned things up quite nicely, but it is by no means complete. We can check all the different place labels used by extracting a list of the unique values of that column. When we do this, we see that there is actually quite a lot more work needed to completely clean up that column.

In [ ]:
pd.Series(df['Place of Publication'].unique())

Let's take a look at a different example file to clean up, `university_towns.txt`. Note that this is a text file with lines of text, and not a CSV file representing a table.

In [ ]:
uni_towns = open('university_towns.txt')
uni_towns.readlines()[:20]

What we can see from looking at the first 20 lines of the file is that we have section labels that are names of US states with the text `[edit]` attached to it, followed by a list of names of university towns with the names of the universities that are situated in each of those towns in parentheses. We can take advantage of this pattern to preprocess the file into a CSV format.

In [ ]:
uni_towns = []
with open('university_towns.txt') as file:
    for line in file:
        if '[edit]' in line:
            state = line
        else:
            uni_towns.append((state, line))
uni_towns[:20]

We can now load this into a DataFrame.

In [ ]:
uni_towns_df = pd.DataFrame(uni_towns, columns=['State', 'RegionName'])
uni_towns_df

We can see now that the cells in our DataFrame need a lot more cleaning up. We could have done this in the loop we used above, but the Pandas library makes it very easy using the `applymap()` function.

First we define a function that can independently clean up one cell's contents.

In [ ]:
def get_citystate(item):
    if ' (' in item:
        return item[:item.find(' (')]
    elif '[' in item:
        return item[:item.find('[')]
    else:
        return item

Let's test this on a couple of example strings.

In [ ]:
get_citystate('Wyoming[edit]\n')

In [ ]:
get_citystate('Eau Claire (University of Wisconsin–Eau Claire)\n')

With `applymap()` we can pass the function name so that it runs the function on all cells in the DataFrame.

In [ ]:
uni_towns_df = uni_towns_df.applymap(get_citystate)
uni_towns_df

Somtimes datasets that you work with will either have column names that are not easy to understand, or unimportant information in the first or last few rows, for example definitions of terms in the dataset or some footnotes. In these cases we can rename columns or drop certain rows.

Let's take a look at another example dataset in `olympics.csv`.

In [ ]:
uni_towns = open('olympics.csv')
uni_towns.readlines()[:20]

It looks like we can load this into a DataFrame straight away, so we will do that.

In [ ]:
olympics_df = pd.read_csv('olympics.csv')
olympics_df.head()

We can see that the top of this dataset is definitely messy! The first row seems to be an index of the columns (0, 1, 2, 3, etc.) and then the row that should be used as the column names looks like when it was written to disk that the special characters were not rendered correctly, and requires renaming across them.

Pandas gives us an easy way to set the correct header. When we use `read_csv()` we can provide an extra parameter to tell it which row to use as the header.

In [ ]:
olympics_df = pd.read_csv('olympics.csv', header=1)
olympics_df.head()

Now, we can rename the column headers using the DataFrame's `rename()` method. This function allows us to rename any axis based on a mapping, which is represented using a Python dictionary (a `dict` datatype).

In [ ]:
name_mapping = {
    'Unnamed: 0': 'Country',
    '? Summer': 'Summer Olympics',
    '01 !': 'Gold',
    '02 !': 'Silver',
    '03 !': 'Bronze',
    '? Winter': 'Winter Olympics',
    '01 !.1': 'Gold.1',
    '02 !.1': 'Silver.1',
    '03 !.1': 'Bronze.1',
    '? Games': '# Games',
    '01 !.2': 'Gold.2',
    '02 !.2': 'Silver.2',
    '03 !.2': 'Bronze.2'
}

In [ ]:
olympics_df = olympics_df.rename(columns=name_mapping)
olympics_df.head()